In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.learn.dann import ResNetForMultiLabel

import numpy as np
import pandas as pd
import torch

In [4]:
a = ResNetForMultiLabel()

TypeError: ResNetForMultiLabel.__init__() missing 3 required positional arguments: 'config', 'num_d1_classes', and 'lamb'

In [ ]:


train_data = br.dataset_load('/data/D21_preprocessed_v2/D21_preprocessed_train_v2.npz')
val_data = br.dataset_load('/data/D21_preprocessed_v2/D21_preprocessed_val_v2.npz')


print(torch.__version__)

# Set device and verify CUDA availability
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("Device count:", torch.cuda.device_count())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

In [6]:
from transformers.models.resnet.modeling_resnet import ResNetForImageClassification


Raw logits for labels:
	 tensor([[-1.1258, -1.1524, -0.2506, -0.4339,  0.8487],
        [ 0.6920, -0.3160, -2.1152,  0.3223, -1.2633],
        [ 0.3500,  0.3081,  0.1198,  1.2377,  1.1168],
        [-0.2473, -1.3527, -1.6959,  0.5667,  0.7935],
        [ 0.5988, -1.5551, -0.3414,  1.8530,  0.7502],
        [-0.5855, -0.1734,  0.1835,  1.3894,  1.5863],
        [ 0.9463, -0.8437, -0.6136,  0.0316,  1.0554],
        [ 0.1778, -0.2303, -0.3918,  0.5433, -0.3952],
        [ 0.2055, -0.4503,  1.5210,  3.4105, -1.5312],
        [-1.2341,  1.8197, -0.5515, -1.3253,  0.1886]], requires_grad=True)
Raw logits for domains:
	 tensor([ 0.6577,  0.9387,  1.1317, -0.6455, -1.7703,  0.2143, -0.5382,  0.5880,
         1.6059,  0.4279], requires_grad=True)
True labels:
	 tensor([4, 3, 4, 3, 0, 3, 2, 3, 0, 1])
True domains:
	 tensor([0, 1, 1, 0, 1, 0, 0, 1, 0, 0])
Gradient of loss w.r.t. label logits:
	 tensor([[ 0.0074,  0.0072,  0.0177,  0.0147, -0.0469],
        [ 0.0000,  0.0000,  0.0000, -0.0000,  0

In [36]:

class InstanceWeightedCrossEntropyLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.base_loss = torch.nn.CrossEntropyLoss(reduction='none')

    def forward(self, logits: torch.Tensor, labels: torch.Tensor, instance_weights: torch.Tensor) -> torch.Tensor:
        """
        Args:
            logits: Shape [N, K]. Model's predicted logits for label y classification.
            labels: Shape [N]. Ground truth class indices.
            instance_weights: Shape [N]. Weights for each instance in the minibatch.
        Returns:
            Shape []. Loss value.
        """
        loss = self.base_loss(logits, labels)
        masked_loss = loss * instance_weights
        reduced_loss = masked_loss.mean()
        return reduced_loss

In [64]:
# k =  torch.nn.BCEWithLogitsLoss()

# a = torch.tensor([1., 2., 3.])
# y = torch.tensor([0., 1., 1.])

# k(a,y)


q = InstanceWeightedCrossEntropyLoss()
b = torch.tensor([[3., 1., 4., 1.,],
                 [1., 5., 2., 6.,]], requires_grad=True)
c = torch.tensor([1, 3])
d = torch.tensor([0., 1.])
l = q(logits=b, labels=c, instance_weights=d)
l.backward()
b.grad

tensor([[ 0.0000, -0.0000,  0.0000,  0.0000],
        [ 0.0024,  0.1321,  0.0066, -0.1410]])

In [68]:
torch.long

torch.int64

tensor(0.4963)

In [17]:
b.shape

torch.Size([1, 3])